In [1]:
import pandas as pd
import re
import requests

from time import sleep
from bs4 import BeautifulSoup

In [2]:
# Extraindo dados de direito trabalhista

def extract_text(text, pattern):
    try:
        found = re.search(pattern, text).group(1)
    except AttributeError:
        found = '0'
    return found

def extract_post(link):
    url  = "https://forumjuridico.org/" + link
    page = requests.get(url, headers={"User-Agent": "XY"})
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find("blockquote", class_="messageText ugc baseHtml").text
    #sleep(1.0)
    return results

# Bloco onde definiremos a quantidade de paginas que vamos buscar dentro do fórum (cada pagina possui 20 perguntas)
# e extraímos as informacoes mais basicas de cada uma (titulo, data, # de visitas, # de respostas e link para a postagem).

for i in range(0, 50):
    
    url  = "https://forumjuridico.org/forums/direito-do-trabalho.17/page-" + str(i) + "?_params=Array"
    page = requests.get(url, headers={"User-Agent": "XY"})

    # Request para extrair o documento html
    
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="content")
    elements = soup.find_all("li", class_="discussionListItem visible")

    # Extração das informações pertinentes dentro de cada pagina
    
    title_lst = []
    date_lst = []
    answer_lst = []
    visit_lst = []
    url_lst = []
    post_lst = []
    
    for element in elements:
        title_lst.append(element.find("h3", class_="title").text.strip())
        date_lst.append(element.find("span", class_="DateTime").text.strip())
        answer_lst.append(extract_text(element.find("dl", class_="major").text.strip(), '.+?(\d)'))
        visit_lst.append(extract_text(element.find("dl", class_="minor").text.strip(), '.+?(\d)'))
        url_lst.append(extract_text(str(element.find("h3", class_="title").find("a", class_="PreviewTooltip")), 
                                   'href=\"(.+?)\"\ title'))
        post_lst.append(extract_post(extract_text(str(element.find("h3", class_="title").find("a", class_="PreviewTooltip")), 
                                   'href=\"(.+?)\"\ title')))
 


trabalhista_df = pd.DataFrame(
    {'title': title_lst,
     'date': date_lst,
     'answers': answer_lst,
     'visits': visit_lst,
     'url': url_lst,
     'post': post_lst
    })

trabalhista_df.head()

,title,date,answers,visits,url,post
0,recurso cabível exceção de preexecutividade re...,11 de Março de 2021,1,1,threads/recurso-cabivel-excecao-de-preexecutiv...,"\n\t\t\t\t\t\n\t\t\t\t\tBoas tardes, caros col..."
1,Liquidez - Reforma Trabalhista,06 de Março de 2021,0,6,threads/liquidez-reforma-trabalhista.23092/,"\n\t\t\t\t\t\n\t\t\t\t\tBoa noite, colegas.\n\..."
2,Rito Sumário - sentença irrecorrível?,06 de Março de 2021,0,6,threads/rito-sumario-sentenca-irrecorrivel.23091/,"\n\t\t\t\t\t\n\t\t\t\t\tOlá, colegas.\n\nAlgué..."
3,DISPENSA DE FUNCIONARIO AFASTADO EM ESPECIE DE...,27 de Janeiro de 2021,0,9,threads/dispensa-de-funcionario-afastado-em-es...,"\n\t\t\t\t\t\n\t\t\t\t\tPrezados, Drs.\n\nPrim..."
4,Cálculo Trabalhista - contador ou advogado?,03 de Maio de 2018,8,1,threads/calculo-trabalhista-contador-ou-advoga...,"\n\t\t\t\t\t\n\t\t\t\t\tQueridos (as),\n\nUm c..."


In [3]:
# Extraindo dados de direito empresarial

def extract_text(text, pattern):
    try:
        found = re.search(pattern, text).group(1)
    except AttributeError:
        found = 0
    return found

def extract_post(link):
    url  = "https://www.perguntedireito.com.br" + link
    page = requests.get(url, headers={"User-Agent": "XY"})
    soup = BeautifulSoup(page.content, "html.parser")
    results = extract_text(str(soup), 'itemprop\=\"text\"\>(.+?)\<\/div')
    #sleep(1.0)
    return results

# Bloco onde definiremos a quantidade de paginas que vamos buscar dentro do fórum (cada pagina possui 10 perguntas)
# e extraímos as informacoes mais basicas de cada uma (titulo, data, # de visitas, # de respostas e link para a postagem).

for i in range(0, 100):
    
    url  = "https://www.perguntedireito.com.br/questions/direito-empresarial?start=" + str(i * 10)
    page = requests.get(url, headers={"User-Agent": "XY"})

    # Request para extrair o documento html

    soup = BeautifulSoup(page.content, "html.parser")
    elements = soup.find_all("div", class_="qa-q-list-item")

    title_lst = []
    date_lst = []
    answer_lst = []
    url_lst = []
    post_lst = []

    for element in elements:
        title_lst.append(element.find("div", class_="qa-q-item-title").text.strip())
        date_lst.append(element.find("span", class_="qa-q-item-when-data").text.strip())
        answer_lst.append(extract_text(element.find("span", class_="qa-a-count-data").text.strip(), '.+?(\d)'))
        url_lst.append(extract_text(str(element.find("div", class_="qa-q-item-title")), 
                                   'href=\"(.+?)\"\>'))
        post_lst.append(
            extract_post(
                extract_text(
                    str(
                        element.find("div", class_="qa-q-item-title")
                    ),'href=\"(.+?)\"\>')
            )
        )

# Transformando as listas em um dataframe e visualizando o resultado
     
empresarial_df = pd.DataFrame(
    {'title': title_lst,
     'date': date_lst,
     'answers': answer_lst,
     'url': url_lst,
     'post': post_lst
    })

empresarial_df.head()

,title,date,answers,url,post
0,Sou professora estadual em SP. Posso abrir MEI...,3 dias,0,/29853/professora-estadual-servicos-educacao-e...,0
1,Professor de São Bernardo do Campo pode ter ME...,em 9 de junho de 2021,0,/29718/professor-de-sao-bernardo-campo-pode-te...,Sou professora da rede de São Bernardo e gosta...
2,Tive 15 falta durante o ano faltas que já fora...,em 10 de maio de 2021,0,/29591/durante-descontadas-salario-empresa-des...,"Descontou nas férias 738,00 equivalente aos 16..."
3,Esposo de vice prefeita não pode fazer parte d...,em 16 de janeiro de 2021,0,/28722/esposo-prefeita-fazer-parte-diretoria-h...,0
4,Por que as empresas públicas e sociedades de e...,em 24 de setembro de 2020,0,/27776/empresas-publicas-sociedades-economia-c...,Estou com essa dúvida e não posso copiar da in...


In [ ]:
# Salvandos os dataframes em um arquivo csv

trabalhista_df.to_csv(sep='||', header=True, index=False)
empresarial_df.to_csv(sep='||', header=True, index=False)